#                OSIC Pulmonary Fibrosis Progression


***1. Introduction***


**1.1 What is Pulmonary fibrosis?**
Idiopathic pulmonary fibrosis (IPF) is a type of chronic scarring lung disease characterized by a progressive and irreversible decline in lung function. Symptoms typically include gradual onset of shortness of breath and a dry cough.Other changes may include feeling tired, and abnormally large and dome shaped finger and toenails (nail clubbing).Complications may include pulmonary hypertension, heart failure, pneumonia, or pulmonary embolism.

The cause is unknown.Risk factors include cigarette smoking, certain viral infections, and a family history of the condition.The underlying mechanism involves scarring of the lungs.Diagnosis requires ruling out other potential causes.It may be supported by a CT scan or lung biopsy which show usual interstitial pneumonia (UIP).It is a type of interstitial lung disease (ILD).

People often benefit from pulmonary rehabilitation and supplemental oxygen.Certain medications like pirfenidone or nintedanib may slow the progression of the disease.Lung transplantation may also be an option.

About 5 million people are affected globally.The disease newly occurs in about 12 per 100,000 people per year.Those in their 60s and 70s are most commonly affected.Males are affected more often than females. Average life expectancy following diagnosis is about four years.
![](https://previews.123rf.com/images/stockdevil/stockdevil1503/stockdevil150300002/37249266-pulmonary-tuberculosis-collection-chest-x-ray-show-patchy-infiltration-interstitial-infiltration-alv.jpg)

**1.2 What is OSIC Pulmonary Fibrosis Progression Competition?**
In this competition, you’ll predict a patient’s severity of decline in lung function based on a CT scan of their lungs. You’ll determine lung function based on output from a spirometer, which measures the volume of air inhaled and exhaled. The challenge is to use machine learning techniques to make a prediction with the image, metadata, and baseline FVC as input.

**1.3 What I need to do? Observation**
I will predict a patient’s severity of decline in lung function based on a CT scan of their lungs. In other words, I will predict the final three FVC measurements for each patient, as well as a confidence value in your prediction.

The leaderboard of this competition is calculated with approximately 1%->15% of the test data. The final results will be based on the other 99%->85%, so the final standings may be different.

**1.4 Metric: Laplace Log Likelihood**

![](https://i.imgur.com/tEIZvli.png)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Laplace_cdf_mod.svg/488px-Laplace_cdf_mod.svg.png)


The evaluation metric of this competition is a modified version of Laplace Log Likelihood. Read more about it on the Evaluation Page.

If you feel this was something new and fresh, and it added some value to you, please consider upvoting, it motivates to keep writing good kernels

**Contents**

Basic Exploratory Data Analysis

Getting started - Importing libraries

Reading the train.csv

Data Exploration

Check Train & Test Info.

Unique Patients(Ids)

Exploring the 'SmokingStatus' column

Weeks distribution

FVC - The forced vital capacity

Exploring the Percent column

Gender Distribution

Patient Overlap

Visualising Images : DECOM

Visualising One DECOM Image & Info

Visualising Multiple DECOM Images

Visualization using gif

Extracting DIOCOM files Info.

Pandas Profiling

Pandas Profiling Report for Train.csv

Pandas Profiling Report for Test.csv



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing the necessary libraries**

In [ ]:
import os
from os import listdir
import pandas as pd
import numpy as np
import glob
import tqdm
from typing import Dict
import matplotlib.pyplot as plt
%matplotlib inline

#plotly
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

#color
from colorama import Fore, Back, Style

import seaborn as sns
sns.set(style="whitegrid")

#pydicom
import pydicom

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Settings for pretty nice plots
plt.style.use('fivethirtyeight')
plt.show()


**Reading the train.csv**

In [ ]:
# List files available
list(os.listdir("../input/osic-pulmonary-fibrosis-progression"))

In [ ]:
from colorama import Fore, Back, Style
IMAGE_PATH = "../input/osic-pulmonary-fibrosis-progressiont/"

train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

print(Fore.BLUE + 'Training data shape: ',Style.RESET_ALL, train_df.shape)
train_df.head(10)

In [ ]:
train_df.groupby(['SmokingStatus']).count()['Sex'].to_frame()

**Basic Data Exploration**


**General Info**

In [ ]:
# Null values and Data types
print(Fore.BLUE + 'Train Set !!',Style.RESET_ALL)
print(train_df.info())
print('-------------')
print(Fore.GREEN + 'Test Set !!',Style.RESET_ALL)
print(test_df.info())

**Missing values**

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

**There is no missing values in train_df and test_df.**

In [ ]:
# Total number of Patient in the dataset(train+test)

print(Fore.BLUE +"Total Patients in Train set: ",Style.RESET_ALL,train_df['Patient'].count())
print(Fore.GREEN +"Total Patients in Test set: ",Style.RESET_ALL,test_df['Patient'].count())

In [ ]:
print(Fore.BLUE + "The total patient ids are",Style.RESET_ALL,f"{train_df['Patient'].count()},", Fore.GREEN + "from those the unique ids are", Style.RESET_ALL, f"{train_df['Patient'].value_counts().shape[0]}.")

In [ ]:
train_patient_ids = set(train_df['Patient'].unique())
test_patient_ids = set(test_df['Patient'].unique())

train_patient_ids.intersection(test_patient_ids)

**We already see 5 patients in test set that can be found in train set as well.**

In [ ]:
columns = train_df.keys()
columns = list(columns)
print(columns)

****Patient Counts****

In [ ]:
train_df['Patient'].value_counts().max()

In [ ]:
test_df['Patient'].value_counts().max()

In [ ]:
np.quantile(train_df['Patient'].value_counts(), 0.75) - np.quantile(test_df['Patient'].value_counts(), 0.25)

In [ ]:
print(np.quantile(train_df['Patient'].value_counts(), 0.95))
print(np.quantile(test_df['Patient'].value_counts(), 0.95))

# Number of Patients and Images in Training Images Folder
 https://www.kaggle.com/yeayates21/osic-simple-image-eda

In [ ]:
files = folders = 0

path = "/kaggle/input/osic-pulmonary-fibrosis-progression/train"

for _, dirnames, filenames in os.walk(path):
  # ^ this idiom means "we won't be using this value"
    files += len(filenames)
    folders += len(dirnames)
#print(Fore.YELLOW +"Total Patients in Train set: ",Style.RESET_ALL,train_df['Patient'].count())
print(Fore.BLUE +f'{files:,}',Style.RESET_ALL,"files/images, " + Fore.GREEN + f'{folders:,}',Style.RESET_ALL ,'folders/patients')

In [ ]:
files = []
for _, dirnames, filenames in os.walk(path):
  # ^ this idiom means "we won't be using this value"
    files.append(len(filenames))

print(Fore.YELLOW +f'{round(np.mean(files)):,}',Style.RESET_ALL,'average files/images per patient')
print(Fore.BLUE +f'{round(np.max(files)):,}',Style.RESET_ALL, 'max files/images per patient')
print(Fore.GREEN +f'{round(np.min(files)):,}',Style.RESET_ALL,'min files/images per patient')

**Data Exploration in Details**

**Individual Patient Dataframe**

**for 175 unique patients, we make new dataframe**

https://www.kaggle.com/redwankarimsony/pulmonary-fibrosis-progression-interactive-eda

In [ ]:
patient_df = train_df[['Patient', 'Age', 'Sex', 'SmokingStatus']].drop_duplicates()
patient_df.head()

In [ ]:
# Creating unique patient lists and their properties. 
train_dir = '../input/osic-pulmonary-fibrosis-progression/train/'
test_dir = '../input/osic-pulmonary-fibrosis-progression/test/'

patient_ids = os.listdir(train_dir)
patient_ids = sorted(patient_ids)

#Creating new rows
no_of_instances = []
age = []
sex = []
smoking_status = []

for patient_id in patient_ids:
    patient_info = train_df[train_df['Patient'] == patient_id].reset_index()
    no_of_instances.append(len(os.listdir(train_dir + patient_id)))
    age.append(patient_info['Age'][0])
    sex.append(patient_info['Sex'][0])
    smoking_status.append(patient_info['SmokingStatus'][0])

#Creating the dataframe for the patient info    
patient_df = pd.DataFrame(list(zip(patient_ids, no_of_instances, age, sex, smoking_status)), 
                                 columns =['Patient', 'no_of_instances', 'Age', 'Sex', 'SmokingStatus'])
print(patient_df.info())
patient_df.head()

In [ ]:
patient_df['SmokingStatus'].value_counts()

**Exploring the 'SmokingStatus' column**

In [ ]:
patient_df['SmokingStatus'].value_counts().iplot(kind='bar',
                                              yTitle='Percentage', 
                                              linecolor='red', 
                                              opacity=0.7,
                                              color='blue',
                                              theme='pearl',
                                              bargap=0.8,
                                              gridcolor='white',
                                              title='Distribution of the SmokingStatus column in the Unique Patient Set')

18 : Ex-smoker

49 : Never smoked

9 : Currently smokes

**Weeks distribution**

In [ ]:
train_df['Weeks'].value_counts().head()

In [ ]:
train_df['Weeks'].value_counts().iplot(kind='barh',
                                      xTitle='Counts(Weeks)', 
                                      linecolor='black', 
                                      opacity=0.7,
                                      color='#450902',
                                      theme='pearl',
                                      bargap=0.2,
                                      gridcolor='white',
                                      title='Distribution of the Weeks in the training set')

In [ ]:
train_df['Weeks'].iplot(kind='hist',
                              xTitle='Counts(Weeks)', 
                              linecolor='black', 
                              opacity=0.7,
                              color='#8072FB',
                              theme='pearl',
                              bargap=0.2,
                              gridcolor='white',
                              title='Distribution of the Weeks in the training set')

There are some negative values for Weeks.

Because Weeks is the relative number of weeks pre/post the baseline CT.

Distribution Age over Week

In [ ]:
fig = px.scatter(train_df, x="Weeks", y="Age", color='Sex')
fig.show()

# FVC - The forced vital capacity
# The forced vital capacity (FVC), i.e. the volume of air exhaled

# the recorded lung capacity in ml

In [ ]:
train_df['FVC'].value_counts()

In [ ]:
train_df['FVC'].iplot(kind='hist',
                      xTitle='Lung Capacity(ml)', 
                      linecolor='black', 
                      opacity=0.8,
                      color='#FB72ED',
                      bargap=0.5,
                      gridcolor='white',
                      title='Distribution of the FVC in the training set')

# FVC vs Percent

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Percent", color='Age')
fig.show()

FVC seems to related Percent linearly.

# FVC vs Age

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Age", color='Sex')
fig.show()

# FVC vs Weeks

In [ ]:
fig = px.scatter(train_df, x="FVC", y="Weeks", color='SmokingStatus')
fig.show()

# Pick one patient for FVC vs Weeks

In [ ]:
patient = train_df[train_df.Patient == 'ID00422637202311677017371']
fig = px.line(patient, x="Weeks", y="FVC", color='SmokingStatus')
fig.show()

Percent
A computed field which approximates the patient's FVC as a percent of the typical FVC for a person of similar characteristics

In [ ]:
train_df['Percent'].value_counts()

In [ ]:
train_df['Percent'].iplot(kind='hist',bins=30,color='#FB7286',xTitle='Percent distribution',yTitle='Count')

# Percent vs SmokingStatus In Patient Dataframe

In [ ]:
df = train_df
fig = px.violin(df, y='Percent', x='SmokingStatus', box=True, color='Sex', points="all",
          hover_data=train_df.columns)
fig.show()

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.violinplot(x = train_df['SmokingStatus'], y = train_df['Percent'], palette = 'Blues')
ax.set_xlabel(xlabel = 'Smoking Habit', fontsize = 15)
ax.set_ylabel(ylabel = 'Percent', fontsize = 15)
ax.set_title(label = 'Distribution of Smoking Status Over Percentage', fontsize = 20)
plt.show()

In [ ]:
df = px.data.iris() # iris is a pandas DataFrame
fig = px.scatter(train_df, x="Age", y="Percent", color='SmokingStatus')
fig.show()

# Age Distribution of Unique Patients

In [ ]:
patient_df['Age'].iplot(kind='hist',bins=30,color='#FB72ED',xTitle='Ages of distribution',yTitle='Count')

# Distribution of Age vs SmokingStatus In Patient Dataframe

In [ ]:
patient_df['SmokingStatus'].value_counts()

In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Ex-smoker', 'Age'], label = 'Ex-smoker',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Never smoked', 'Age'], label = 'Never smoked',shade=True)
sns.kdeplot(patient_df.loc[patient_df['SmokingStatus'] == 'Currently smokes', 'Age'], label = 'Currently smokes', shade=True)

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.violinplot(x = patient_df['SmokingStatus'], y = patient_df['Age'], palette = 'Blues')
ax.set_xlabel(xlabel = 'Smoking habit', fontsize = 15)
ax.set_ylabel(ylabel = 'Age', fontsize = 15)
ax.set_title(label = 'Distribution of Smokers over Age', fontsize = 20)
plt.show()


In [ ]:
plt.figure(figsize=(16, 6))
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Male', 'Age'], label = 'Male',shade=True)
sns.kdeplot(patient_df.loc[patient_df['Sex'] == 'Female', 'Age'], label = 'Female',shade=True)
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

In [ ]:
patient_df['Sex'].value_counts()

In [ ]:
patient_df['Sex'].value_counts().iplot(kind='bar',
                                          yTitle='Count', 
                                          linecolor='black', 
                                          opacity=0.7,
                                          color='#320601',
                                          theme='pearl',
                                          bargap=0.8,
                                          gridcolor='white',
                                          title='Distribution of the Sex column in Patient Dataframe')

In [ ]:
plt.figure(figsize=(16, 6))
a = sns.countplot(data=patient_df, x='SmokingStatus', hue='Sex')

for p in a.patches:
    a.annotate(format(p.get_height(), ','), 
           (p.get_x() + p.get_width() / 2., 
            p.get_height()), ha = 'center', va = 'center', 
           xytext = (0, 4), textcoords = 'offset points')

plt.title('Gender split by SmokingStatus', fontsize=16)
sns.despine(left=True, bottom=True);

In [ ]:
df = px.data.tips()
fig = px.box(patient_df, x="Sex", y="Age", points="all")
fig.show()

In [ ]:
# Extract patient id's for the training set
ids_train = train_df.Patient.values
# Extract patient id's for the validation set
ids_test = test_df.Patient.values

# Create a "set" datastructure of the training set id's to identify unique id's
ids_train_set = set(ids_train)
print(f'There are {len(ids_train_set)} unique Patient IDs in the training set')
# Create a "set" datastructure of the validation set id's to identify unique id's
ids_test_set = set(ids_test)
print(f'There are {len(ids_test_set)} unique Patient IDs in the test set')

# Identify patient overlap by looking at the intersection between the sets
patient_overlap = list(ids_train_set.intersection(ids_test_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and test sets')
print('')
print(f'These patients are in both the training and test datasets:')
print(f'{patient_overlap}')

# Heatmap for train.csv

In [ ]:
corrmat = train_df.corr() 
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax = ax, cmap = 'RdPu', linewidths = 0.5) 

In [ ]:
print('Train .dcm number of images:', len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/train'))), '\n' +
      'Test .dcm number of images:', len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/test'))), '\n' +
      '--------------------------------', '\n' +
      'There is the same number of images as in train/ test .csv datasets')

In [ ]:
def plot_pixel_array(dataset, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(dataset.pixel_array, cmap='gray') # cmap=plt.cm.bone)
    plt.show()

In [ ]:
# https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data
def show_dcm_info(dataset):
    print("Filename.........:", file_path)

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    
    print(dataset.data_element("ImageOrientationPatient"))
    print(dataset.data_element("ImagePositionPatient"))
    print(dataset.data_element("PatientID"))
    print(dataset.data_element("PatientName"))
    print(dataset.data_element("PatientSex"))
   
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

In [ ]:
i = 1
num_to_plot = 2
for folder_name in os.listdir('../input/osic-pulmonary-fibrosis-progression/train/'):
        patient_path = os.path.join('../input/osic-pulmonary-fibrosis-progression/train/',folder_name)
        
        for i in range(1, num_to_plot+1):     
            file_path = os.path.join(patient_path, str(i) + '.dcm')

            dataset = pydicom.dcmread(file_path)
            show_dcm_info(dataset)
            plot_pixel_array(dataset)

        break

In [ ]:
#https://www.kaggle.com/yeayates21/osic-simple-image-eda

imdir = "/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00123637202217151272140"
print("total images for patient ID00123637202217151272140: ", len(os.listdir(imdir)))

# view first (columns*rows) images in order
w=10
h=10
fig=plt.figure(figsize=(12, 12))
columns = 5
rows = 5
imglist = os.listdir(imdir)
for i in range(1, columns*rows +1):
    filename = imdir + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='nipy_spectral')
plt.show()

In [ ]:
# https://www.kaggle.com/yeayates21/osic-simple-image-eda

imdir = "/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00123637202217151272140"
print("total images for patient ID00123637202217151272140: ", len(os.listdir(imdir)))

# view first (columns*rows) images in order
w=10
h=10
fig=plt.figure(figsize=(12, 12))
columns = 4
rows = 5
imglist = os.listdir(imdir)
for i in range(1, columns*rows +1):
    filename = imdir + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='gnuplot')
plt.show()

In [ ]:
apply_resample = False

def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [ ]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
def set_lungwin(img, hu=[-1200., 600.]):
    lungwin = np.array(hu)
    newimg = (img-lungwin[0]) / (lungwin[1]-lungwin[0])
    newimg[newimg < 0] = 0
    newimg[newimg > 1] = 1
    newimg = (newimg * 255).astype('uint8')
    return newimg

In [ ]:
scans = load_scan('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/')
scan_array = set_lungwin(get_pixels_hu(scans))

In [ ]:
scans = load_scan('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/')
scan_array = set_lungwin(get_pixels_hu(scans))

In [ ]:
# Resample to 1mm (An optional step, it may not be relevant to this competition because of the large slice thickness on the z axis)

from scipy.ndimage.interpolation import zoom

def resample(imgs, spacing, new_spacing):
    new_shape = np.round(imgs.shape * spacing / new_spacing)
    true_spacing = spacing * imgs.shape / new_shape
    resize_factor = new_shape / imgs.shape
    imgs = zoom(imgs, resize_factor, mode='nearest')
    return imgs, true_spacing, new_shape

spacing_z = (scans[-1].ImagePositionPatient[2] - scans[0].ImagePositionPatient[2]) / len(scans)

if apply_resample:
    scan_array_resample = resample(scan_array, np.array(np.array([spacing_z, *scans[0].PixelSpacing])), np.array([1.,1.,1.]))[0]

In [ ]:
import imageio
from IPython.display import Image

imageio.mimsave("/tmp/gif.gif", scan_array, duration=0.0002)
Image(filename="/tmp/gif.gif", format='png')

In [ ]:
from matplotlib.widgets import Slider
import matplotlib.animation as animation
from IPython.display import HTML

fig = plt.figure()

ims = []
for image in scan_array:
    im = plt.imshow(image, animated=True, cmap="Greys")
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
?np.sort

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
patient_id = "ID00035637202182204917484"

dicom_path = "/kaggle/input/osic-pulmonary-fibrosis-progression/train"

files = np.array([f.replace(".dcm","") for f in os.listdir(f"{dicom_path}/{patient_id}/")])
files = -np.sort(-files.astype("int"))
dicoms = [f"{dicom_path}/{patient_id}/{f}.dcm" for f in files]

In [ ]:
images = []
for dcm in dicoms:
    tmp = pydicom.dcmread(dcm)
    slope = tmp.RescaleSlope
    intercept = tmp.RescaleIntercept
    final = tmp.pixel_array*slope + intercept
    images.append(final)
    
images = np.array(images) 

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[0],10):
    im = plt.imshow(images[image,:,:], 
                    animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[1],5):
    im = plt.imshow(images[:,image,:], animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

HTML(ani.to_jshtml())

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[2],5):
    im = plt.imshow(images[:,:,image], animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

HTML(ani.to_jshtml())

In [ ]:
plt.hist(np.array(images).reshape(-1,), bins=50)
plt.show()

In [ ]:
import matplotlib.animation as animation

fig = plt.figure()

ims = []
for image in scan_array:
    im = plt.imshow(image, animated=True, cmap="Greys")
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
HTML(ani.to_html5_video())

# Extracting DIOCOM files information in a dataframe

In [ ]:
from typing import Dict
def extract_dicom_meta_data(filename: str) -> Dict:
    # Load image
    
    image_data = pydicom.read_file(filename)
    img=np.array(image_data.pixel_array).flatten()
    row = {
        'Patient': image_data.PatientID,
        'body_part_examined': image_data.BodyPartExamined,
        'image_position_patient': image_data.ImagePositionPatient,
        'image_orientation_patient': image_data.ImageOrientationPatient,
        'photometric_interpretation': image_data.PhotometricInterpretation,
        'rows': image_data.Rows,
        'columns': image_data.Columns,
        'pixel_spacing': image_data.PixelSpacing,
        'window_center': image_data.WindowCenter,
        'window_width': image_data.WindowWidth,
        'modality': image_data.Modality,
        'StudyInstanceUID': image_data.StudyInstanceUID,
        'SeriesInstanceUID': image_data.StudyInstanceUID,
        'StudyID': image_data.StudyInstanceUID, 
        'SamplesPerPixel': image_data.SamplesPerPixel,
        'BitsAllocated': image_data.BitsAllocated,
        'BitsStored': image_data.BitsStored,
        'HighBit': image_data.HighBit,
        'PixelRepresentation': image_data.PixelRepresentation,
        'RescaleIntercept': image_data.RescaleIntercept,
        'RescaleSlope': image_data.RescaleSlope,
        'img_min': np.min(img),
        'img_max': np.max(img),
        'img_mean': np.mean(img),
        'img_std': np.std(img)}

    return row

In [ ]:
import glob
import tqdm
train_image_path = '/kaggle/input/osic-pulmonary-fibrosis-progression/train'
train_image_files = glob.glob(os.path.join(train_image_path, '*', '*.dcm'))

meta_data_df = []
for filename in tqdm.tqdm(train_image_files):
    try:
        meta_data_df.append(extract_dicom_meta_data(filename))
    except Exception as e:
        continue

In [ ]:
# Convert to a pd.DataFrame from dict
meta_data_df = pd.DataFrame.from_dict(meta_data_df)
meta_data_df.head()

In [ ]:
# source: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/154658
folder='train'
PATH='../input/osic-pulmonary-fibrosis-progression/'

last_index = 2

column_names = ['image_name', 'dcm_ImageOrientationPatient', 
                'dcm_ImagePositionPatient', 'dcm_PatientID',
                'dcm_PatientName', 'dcm_PatientSex'
                'dcm_rows', 'dcm_columns']

def extract_DICOM_attributes(folder):
    patients_folder = list(os.listdir(os.path.join(PATH, folder)))
    df = pd.DataFrame()
    
    i = 0
    
    for patient_id in patients_folder:
   
        img_path = os.path.join(PATH, folder, patient_id)
        
        print(img_path)
        
        images = list(os.listdir(img_path))
        
        #df = pd.DataFrame()

        for image in images:
            image_name = image.split(".")[0]

            dicom_file_path = os.path.join(img_path,image)
            dicom_file_dataset = pydicom.read_file(dicom_file_path)
                
            '''
            print(dicom_file_dataset.dir("pat"))
            print(dicom_file_dataset.data_element("ImageOrientationPatient"))
            print(dicom_file_dataset.data_element("ImagePositionPatient"))
            print(dicom_file_dataset.data_element("PatientID"))
            print(dicom_file_dataset.data_element("PatientName"))
            print(dicom_file_dataset.data_element("PatientSex"))
            '''
            
            imageOrientationPatient = dicom_file_dataset.ImageOrientationPatient
            #imagePositionPatient = dicom_file_dataset.ImagePositionPatient
            patientID = dicom_file_dataset.PatientID
            patientName = dicom_file_dataset.PatientName
            patientSex = dicom_file_dataset.PatientSex
        
            rows = dicom_file_dataset.Rows
            cols = dicom_file_dataset.Columns
            
            #print(rows)
            #print(columns)
            
            temp_dict = {'image_name': image_name, 
                                    'dcm_ImageOrientationPatient': imageOrientationPatient,
                                    #'dcm_ImagePositionPatient':imagePositionPatient,
                                    'dcm_PatientID': patientID, 
                                    'dcm_PatientName': patientName,
                                    'dcm_PatientSex': patientSex,
                                    'dcm_rows': rows,
                                    'dcm_columns': cols}


            df = df.append([temp_dict])
            
        i += 1
        
        if i == last_index:
            break
            
    return df

In [ ]:
extract_DICOM_attributes('train')

In [ ]:
import pandas_profiling as pdp

In [ ]:
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
profile_train_df = pdp.ProfileReport(train_df)

In [ ]:
profile_train_df

In [ ]:
profile_test_df = pdp.ProfileReport(test_df)

In [ ]:
profile_test_df